In [1]:
!pip install openai

In [6]:
!pip install --upgrade openai

In [1]:
import openai
import json

In [2]:
with open("finetune.json", "r") as f:
    train_data = json.load(f)

In [3]:
train_data

[{'prompt': 'What type of music do you like?',
  'completion': 'I like to listen to pop music and classical music. How about you?'},
 {'prompt': 'What kind of exercise do you think is the healthiest?',
  'completion': 'I think swimming and jogging are among the healthiest forms of exercise. How about you?'},
 {'prompt': 'What countries have you traveled to?',
  'completion': 'I have traveled to countries such as France, Italy, Japan, and the United States. How about you?'},
 {'prompt': 'What are your hobbies and interests?',
  'completion': 'I enjoy reading, traveling, and listening to music. How about you?'},
 {'prompt': 'What do you think is the best way to learn a foreign language?',
  'completion': 'I think learning a foreign language requires listening, reading, speaking, and writing, as well as interacting with people who speak the language as their native tongue. How about you?'},
 {'prompt': "What is a good movie you've seen recently?",
  'completion': 'I recently watched a cla

In [4]:
FINE_TUNED_MODEL = "text-davinci-003"

In [5]:
!openai tools fine_tunes.prepare_data -f finetune.json -q

Analyzing...

- Your file contains 10 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `?`
- All prompts start with prefix `What `
- All completions end with suffix `. How about you?`. This suffix seems very long. Consider replacing with a shorter suffix, such as `\n`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `finetune_prepared (1).jsonl`
Feel free to take a look!

Now use that file when fine-tunin

In [8]:
!openai api fine_tunes.create -t "finetune_prepared (1).jsonl" --batch_size 16

Uploaded file from finetune_prepared (1).jsonl: file-kk4yJUWgqo83oyTtSfROoxzE
Created fine-tune: ft-qTDncSZcy3wVGQkiHSvUoiak
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-13 15:23:26] Created fine-tune: ft-qTDncSZcy3wVGQkiHSvUoiak

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-qTDncSZcy3wVGQkiHSvUoiak




Upload progress:   0%|          | 0.00/1.72k [00:00<?, ?it/s]
Upload progress: 100%|##########| 1.72k/1.72k [00:00<00:00, 1.56Mit/s]


In [10]:
!openai api fine_tunes.follow -i ft-wtZMNV8B7WcifsTlSvmJjv2r

'openai' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!openai api fine_tunes.list

'openai' is not recognized as an internal or external command,
operable program or batch file.


Using the fine-tuned models

In [12]:
ft_qa = "curie:ft-pecu-2023-03-09-18-23-42"
def apply_ft_qa_answer(context, question, answering_model):
    prompt = f"{context}\nQuestion: {question}\nAnswer:"
    result = openai.Completion.create(model=answering_model, prompt=prompt, max_tokens=30, temperature=0, top_p=1, n=1, stop=['.','\n'])
    return result['choices'][0]['text']

In [13]:
context = 'Christmas is an annual festival commemorating the birth of Jesus Christ, observed primarily on December 25 as a religious and cultural celebration among billions of people around the world.'
question = 'What is the most popular holiday?'

In [14]:
apply_ft_qa_answer(context, question, ft_qa)

' Christmas is the most popular holiday in the world'